In [1]:
import pandas as pd
import numpy as np
import folium

import requests
from urllib.parse import urlparse, quote

with open('kakaoapikey.txt',mode='r') as key_file:
    kakao_key = key_file.read()

파일읽어오기

In [2]:
park = pd.read_csv("서울시 주요 공원현황(수정).csv")
park = park[['공원명','면적','공원주소']]
park.head()

,공원명,면적,공원주소
0,남산도시자연공원,2896887㎡ 2454140㎡ 442747㎡,서울특별시 중구 삼일대로 231(예장동)
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원)
2,서울대공원,9132690㎡,경기도 과천시 대공원광장로 102
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가)
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원)


좌표 받아오기

In [3]:
# 이런 경우 때문에 예외처리
park.loc[21]

공원명                         낙산근린공원
면적                       201779㎡  
공원주소    서울특별시 종로구 낙산길 41 (우) 03085
Name: 21, dtype: object

In [4]:
park['공원주소'][21].split(" (우) ")

['서울특별시 종로구 낙산길 41', '03085']

In [5]:
local_url = "https://dapi.kakao.com/v2/local/search/address.json"

lng_list = []
lat_list = []

for i in park.index:
    url = f'{local_url}?query={quote(park["공원주소"][i])}'
    result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
    try : 
        lng_list.append(result['documents'][0]['x'])
        lat_list.append(result['documents'][0]['y'])
    except Exception :
        new = park['공원주소'][i].split(" (우) ")[0]
        url = f'{local_url}?query={quote(new)}'
        result = requests.get(url, headers = {"Authorization": f"KakaoAK {kakao_key}"}).json()
        lng_list.append(result['documents'][0]['x'])
        lat_list.append(result['documents'][0]['y'])


park['위도'] = lat_list
park['경도'] = lng_list

park

,공원명,면적,공원주소,위도,경도
0,남산도시자연공원,2896887㎡ 2454140㎡ 442747㎡,서울특별시 중구 삼일대로 231(예장동),37.5556326132925,126.992217614712
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),37.5390660422084,127.153328745828
2,서울대공원,9132690㎡,경기도 과천시 대공원광장로 102,37.4364305503019,127.014098361931
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가),37.5430701468405,127.041799099222
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원),37.5723265509352,126.881888633222
...,...,...,...,...,...
127,용마도시자연공원(사가정공원),109635㎡,서울 중랑구 면목동 산 50-15,37.5802640273795,127.094512194358
128,문화비축기지,140022㎡,서울특별시 마포구 증산로 87,37.5703466887615,126.895031420115
129,경춘선숲길,211392㎡,서울 노원구 공릉동 706,37.6304565957753,127.072103322862
130,율현공원,157535㎡,서울특별시 강남구 율현동 밤고개로24길(율현동 56),37.4721971560599,127.106177680728


In [6]:
# 혹시 몰라 한 번 저장
park.to_csv('park.csv')

면적처리

In [7]:
# 결측치가 3개 있는 것으로 판단됨
park.면적.isna().sum()

3

In [8]:
# 일단 0으로 채우고 추후에 평균값으로 대체
park.면적.fillna("0",inplace=True)

In [9]:
# 여러개 면적을 가졌다면, 첫번째만 가져오고, 단위 지우기
for i in park.index:
    park.면적[i] = park.면적[i].split()[0]
    park.면적[i] = float(park.면적[i].replace("㎡",""))
park.면적

0      2896887.0
1        80683.0
2      9132690.0
3       480994.0
4      2284085.0
         ...    
127     109635.0
128     140022.0
129     211392.0
130     157535.0
131          0.0
Name: 면적, Length: 132, dtype: object

In [10]:
# 결측치 처리
for i in park.index:
    if park.면적[i] == 0.0:
        park.면적[i] = round(park.면적.mean(),2)
park.tail()

,공원명,면적,공원주소,위도,경도
127,용마도시자연공원(사가정공원),109635.0,서울 중랑구 면목동 산 50-15,37.5802640273795,127.094512194358
128,문화비축기지,140022.0,서울특별시 마포구 증산로 87,37.5703466887615,126.895031420115
129,경춘선숲길,211392.0,서울 노원구 공릉동 706,37.6304565957753,127.072103322862
130,율현공원,157535.0,서울특별시 강남구 율현동 밤고개로24길(율현동 56),37.4721971560599,127.106177680728
131,서울로7017,716740.78,서울 중구 퇴계로 33,37.5575463434632,126.976607882008


원 크기 조절을 위해 그룹화

In [11]:
park['group'] = pd.qcut(park.면적,3,labels=[3,7,10])
park.head()

,공원명,면적,공원주소,위도,경도,group
0,남산도시자연공원,2896887.0,서울특별시 중구 삼일대로 231(예장동),37.5556326132925,126.992217614712,10
1,길동생태공원,80683.0,서울특별시 강동구 천호대로 1291(길동생태공원),37.5390660422084,127.153328745828,7
2,서울대공원,9132690.0,경기도 과천시 대공원광장로 102,37.4364305503019,127.014098361931,10
3,서울숲,480994.0,서울특별시 성동구 뚝섬로 273 (성수동1가),37.5430701468405,127.041799099222,10
4,월드컵공원,2284085.0,서울특별시 마포구 하늘공원로 84(월드컵공원),37.5723265509352,126.881888633222,10


지도위에 그리기

In [12]:
# 기준점을 공원의 위치의 평균으로 설정하기
map = folium.Map(location=[park.위도.astype(float).mean(),park.경도.astype(float).mean()],zoom_start=12)

for i in park.index:

    folium.CircleMarker(location=[park['위도'][i],park['경도'][i]],
                    radius = int(park['group'][i]),
                    popup = folium.Popup(park['공원주소'][i],min_width=220,max_width=220),
                    tooltip=park['공원명'][i],
                    color='green',
                    fill_color=True).add_to(map)
map